In [ ]:
!pip install keras-tuner -q

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
import time
import keras_tuner
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import AdamW


#Mount Google Drive
drive.mount('/content/drive')

#A function to set random seeds for reproducibility
def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()
print("TensorFlow Version:", tf.__version__)
print("KerasTuner Version:", keras_tuner.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.6 MB/s eta 0:00:00
Mounted at /content/drive
TensorFlow Version: 2.19.0
KerasTuner Version: 1.4.7


In [ ]:
#Key parameters for the experiment
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASSES = ["acne", "pigmentation", "wrinkles"]
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/"

#Constructs the full, absolute path for each image file
df = pd.read_csv(os.path.join(DATA_ROOT, "labels.csv"))
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

PATCH_SIZE = 32
NUM_PATCHES = (IMG_SIZE[0] // PATCH_SIZE) ** 2

In [ ]:
#Splitting data into training, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[CLASSES])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df[CLASSES])

#Calculating class counts for the weighted loss function
pos_counts = train_df[CLASSES].sum().values
total_train_samples = len(train_df)

print(f"Train samples: {len(train_df)}, Val samples: {len(val_df)}, Test samples: {len(test_df)}")
print(f"Positive class counts in train set: {pos_counts}")

Train samples: 3656, Val samples: 646, Test samples: 760
Positive class counts in train set: [1015  386  738]


In [ ]:
#Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.2),
], name="data_augmentation")

#Create tf.data Pipelines
def parse_function(filename, labels):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    image_resized = tf.image.resize(image, IMG_SIZE)
    return image_resized, labels

def create_dataset(df, batch_size, augment=False, cache_file=None):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df["filename"].values, df[CLASSES].values.astype(np.float32))
    )
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                                  num_parallel_calls=tf.data.AUTOTUNE)

    #Use disk caching to prevent RAM crashes
    if cache_file:
        dataset = dataset.cache(cache_file)
    else:
        dataset = dataset.cache()

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

#Create the datasets with disk caching
train_cache_file = os.path.join(DATA_ROOT, 'train_cache_eanet')
val_cache_file = os.path.join(DATA_ROOT, 'val_cache_eanet')

train_ds = create_dataset(train_df, BATCH_SIZE, augment=True, cache_file=train_cache_file)
val_ds = create_dataset(val_df, BATCH_SIZE, augment=False, cache_file=val_cache_file)
test_ds = create_dataset(test_df, BATCH_SIZE, augment=False)

print("tf.data pipelines created successfully with disk caching enabled.")

tf.data pipelines created successfully with disk caching enabled.


In [ ]:
class PatchExtractor(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

class ExternalAttention(layers.Layer):
    def __init__(self, dim, num_heads, dim_coefficient=4, attention_dropout=0.2, projection_dropout=0.2, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.num_heads = num_heads
        self.dim_coefficient = dim_coefficient
        self.linear_q = layers.Dense(dim * dim_coefficient)
        self.linear_k = layers.Dense(dim * dim_coefficient)
        self.linear_v = layers.Dense(dim * dim_coefficient)
        self.linear_out = layers.Dense(dim)
        self.softmax = layers.Softmax(axis=-1)
        self.attention_drop = layers.Dropout(attention_dropout)
        self.projection_drop = layers.Dropout(projection_dropout)

    def call(self, inputs, training=False):
        q = self.linear_q(inputs)
        k = self.linear_k(inputs)
        v = self.linear_v(inputs)

        q = tf.reshape(q, (-1, tf.shape(inputs)[1], self.num_heads, self.dim_coefficient))
        k = tf.reshape(k, (-1, tf.shape(inputs)[1], self.num_heads, self.dim_coefficient))
        v = tf.reshape(v, (-1, tf.shape(inputs)[1], self.num_heads, self.dim_coefficient))

        q = tf.transpose(q, perm=[0, 2, 1, 3])
        k = tf.transpose(k, perm=[0, 2, 1, 3])
        v = tf.transpose(v, perm=[0, 2, 1, 3])

        attention = self.softmax(tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(float(self.dim_coefficient)))
        attention = self.attention_drop(attention, training=training)

        out = tf.matmul(attention, v)
        out = tf.transpose(out, perm=[0, 2, 1, 3])
        out = tf.reshape(out, (-1, tf.shape(inputs)[1], self.dim * self.dim_coefficient))

        out = self.linear_out(out)
        out = self.projection_drop(out, training=training)
        return out

    def get_config(self):
      config = super().get_config()
      config.update({
          "dim": self.dim,
          "num_heads": self.num_heads,
          "dim_coefficient": self.dim_coefficient,
          "attention_dropout": self.attention_drop.rate,
          "projection_dropout": self.projection_drop.rate
      })
      return config

class TransformerBlock(layers.Layer):
    def __init__(self, embedding_dim, num_heads, mlp_dim, drop_rate=0.2, attn_drop=0.2, **kwargs):
        super().__init__(**kwargs)
        self.attention = ExternalAttention(
            dim=embedding_dim,
            num_heads=num_heads,
            attention_dropout=attn_drop,
            projection_dropout=drop_rate
        )
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.mlp = tf.keras.Sequential([
            layers.Dense(mlp_dim, activation=tf.nn.gelu),
            layers.Dropout(drop_rate),
            layers.Dense(embedding_dim),
            layers.Dropout(drop_rate)
        ], name="mlp")

        self.dropout1 = layers.Dropout(0.2)
        self.dropout2 = layers.Dropout(0.2)

    def call(self, x, training=False):
        attn_output = self.attention(self.norm1(x), training=training)
        x = x + self.dropout1(attn_output, training=training)
        mlp_output = self.mlp(self.norm2(x), training=training)
        x = x + self.dropout2(mlp_output, training=training)
        return x

    def get_config(self):
        config = super().get_config()
        config.update({
            "embedding_dim": self.attention.dim,
            "num_heads": self.attention.num_heads,
            "mlp_dim": self.mlp.layers[0].units,
            "drop_rate": self.mlp.layers[1].rate,
            "attn_drop": self.attention.attention_drop.rate
        })
        return config


def build_hyper_model(hp):
    """
    Builds the EANet model with hyperparameters for tuning.
    """
    #Defining Hyperparameters to Tune
    EMBEDDING_DIM = hp.Choice('embedding_dim', [192, 256])
    NUM_HEADS = hp.Choice('num_heads', [4, 8])
    NUM_TRANSFORMER_BLOCKS = hp.Int('num_blocks', 3, 6)

    mlp_dim_factor = hp.Choice('mlp_dim_factor', [2, 4])
    MLP_DIM = EMBEDDING_DIM * mlp_dim_factor

    #Tuning dropout rates
    proj_drop = hp.Float('proj_drop', 0.1, 0.4, step=0.1) # After embedding
    transformer_drop_rate = hp.Float('transformer_drop_rate', 0.2, 0.5, step=0.1)
    transformer_attn_drop = hp.Float('transformer_attn_drop', 0.2, 0.5, step=0.1)
    final_drop = hp.Float('final_dropout', 0.3, 0.6, step=0.1) # Before head

    #Model Building
    tf.keras.backend.clear_session()

    inputs = layers.Input(shape=IMG_SIZE + (3,))
    patches = PatchExtractor(PATCH_SIZE)(inputs)
    patch_dims = patches.shape[-1]
    patches = layers.Reshape((NUM_PATCHES, patch_dims))(patches)
    patch_embedding = layers.Dense(units=EMBEDDING_DIM)(patches)

    positions = tf.range(start=0, limit=NUM_PATCHES, delta=1)
    position_embedding = layers.Embedding(
        input_dim=NUM_PATCHES, output_dim=EMBEDDING_DIM
    )(positions)

    x = patch_embedding + position_embedding
    x = layers.Dropout(proj_drop)(x, training=True)

    transformer_layers = []
    for _ in range(NUM_TRANSFORMER_BLOCKS):
        transformer_layers.append(
            TransformerBlock(
                embedding_dim=EMBEDDING_DIM,
                num_heads=NUM_HEADS,
                mlp_dim=MLP_DIM,
                drop_rate=transformer_drop_rate,
                attn_drop=transformer_attn_drop
            )
        )

    transformer_backbone = tf.keras.Sequential(transformer_layers, name="transformer_backbone")
    x = transformer_backbone(x)

    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(final_drop)(x, training=True)

    outputs = layers.Dense(len(CLASSES), activation="sigmoid", dtype='float32')(x)
    model = Model(inputs=inputs, outputs=outputs, name="eanet_hyper_model")

    return model

    #Test build
    hp = keras_tuner.HyperParameters()
    model = build_hyper_model(hp)
    model.summary()

In [ ]:
def create_weighted_bce_loss(pos_counts, total_samples, smooth=0.05):
    """
    A factory function that creates a weighted BCE loss function.
    """
    #These are calculated ONCE and captured by the inner function
    pos = tf.constant(pos_counts, dtype=tf.float32)
    neg = total_samples - pos
    w_pos = neg / tf.maximum(pos, 1.0)
    w_neg = tf.ones_like(pos)

    #This is the actual loss function that will be used
    def weighted_bce(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        #Apply smoothing
        y_true = y_true * (1.0 - smooth) + 0.5 * smooth

        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)

        #Apply weights
        weights = y_true * w_pos + (1.0 - y_true) * w_neg
        return tf.reduce_mean(bce * weights)

    return weighted_bce

#Creating the loss function instance and custom_objects dict

loss_fn = create_weighted_bce_loss(pos_counts, total_train_samples)

#Custom Objects needed for loading/saving
custom_objects = {
    "PatchExtractor": PatchExtractor,
    "TransformerBlock": TransformerBlock,
    "ExternalAttention": ExternalAttention,
    "weighted_bce": loss_fn
}

In [ ]:
class CustomTuner(keras_tuner.RandomSearch):
    """
    Custom Tuner to implement the two-stage training logic.
    """
    def __init__(self, loss_function, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_function = loss_function

    def run_trial(self, trial, train_ds, val_ds, **kwargs):
        hp = trial.hyperparameters
        model = self.hypermodel.build(hp)

        #Define ALL Metrics
        all_metrics = [
            tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
            tf.keras.metrics.AUC(name="auc", multi_label=True),
            tf.keras.metrics.Precision(name="precision", thresholds=0.5),
            tf.keras.metrics.Recall(name="recall", thresholds=0.5),
        ]

        #STAGE 1: FEATURE EXTRACTION (Train Head Only)
        print(f"\n[Trial {trial.trial_id}] Stage 1: Training the classification head...")
        head_lr = hp.Float('head_lr', 1e-4, 1e-3, sampling='log')

        model.get_layer("transformer_backbone").trainable = False
        model.compile(
            optimizer=AdamW(learning_rate=head_lr, weight_decay=1e-4),
            loss=self.loss_function,
            metrics=all_metrics
        )

        model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=10,
            verbose=1
        )

        #STAGE 2: FINE-TUNING
        print(f"\n[Trial {trial.trial_id}] Stage 2: Fine-tuning the entire model...")
        finetune_lr = hp.Float('finetune_lr', 1e-6, 5e-5, sampling='log')

        model.get_layer("transformer_backbone").trainable = True

        model.compile(
            optimizer=AdamW(learning_rate=finetune_lr, weight_decay=1e-4),
            loss=self.loss_function,
            metrics=all_metrics
        )

        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor="val_auc",
                mode="max",
                patience=5,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor="val_auc",
                mode="max",
                factor=0.2,
                patience=3
            )
        ]

        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=70,
            callbacks=callbacks,
            initial_epoch=10,
            verbose=1
        )

        #Report Results
        print(f"[Trial {trial.trial_id}] Evaluating best weights on val_ds...")
        eval_results = model.evaluate(
            val_ds,
            return_dict=True,
            verbose=0,
        )

        val_results_with_prefix = {f"val_{k}": v for k, v in eval_results.items()}

        return val_results_with_prefix

In [ ]:
#Define the tuner
tuner = CustomTuner(
    loss_function=loss_fn,
    hypermodel=build_hyper_model,
    objective=keras_tuner.Objective("val_auc", direction="max"), #Target metric
    max_trials=5,  #How many different models to test
    executions_per_trial=1, #How many times to train each model
    directory=os.path.join(DATA_ROOT, 'keras_tuner'),
    project_name='eanet_skin_tuning',
    overwrite=True
)

#Print a summary of the search space
tuner.search_space_summary()

#Starting the search
print("\nStarting hyperparameter search...")
start_time = time.time()

tuner.search(
    train_ds=train_ds,
    val_ds=val_ds
)

end_time = time.time()
print(f"\nTotal search time: {(end_time - start_time) / 60:.2f} minutes")

Trial 5 Complete [00h 16m 11s]
val_auc: 0.9004563689231873

Best val_auc So Far: 0.9004563689231873
Total elapsed time: 01h 21m 51s

Total search time: 81.84 minutes


In [ ]:
# Show the top 3 performing trials
print("Top 3 Trials")
tuner.results_summary(num_trials=5)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nBest Hyperparameters Found")
print(f"Embedding Dim: {best_hps.get('embedding_dim')}")
print(f"Num Blocks: {best_hps.get('num_blocks')}")
print(f"Num Heads: {best_hps.get('num_heads')}")
print(f"MLP Factor: {best_hps.get('mlp_dim_factor')}")
print(f"Proj Dropout: {best_hps.get('proj_drop'):.3f}")
print(f"Transformer Drop: {best_hps.get('transformer_drop_rate'):.3f}")
print(f"Transformer Attn Drop: {best_hps.get('transformer_attn_drop'):.3f}")
print(f"Final Dropout: {best_hps.get('final_dropout'):.3f}")
print(f"Head LR: {best_hps.get('head_lr'):.1e}")
print(f"Finetune LR: {best_hps.get('finetune_lr'):.1e}")

print("\nBest hyperparameters successfully retrieved.")

Top 3 Trials
Results summary
Results in /content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/keras_tuner/eanet_skin_tuning
Showing 5 best trials
Objective(name="val_auc", direction="max")

Trial 4 summary
Hyperparameters:
embedding_dim: 256
num_heads: 8
num_blocks: 6
mlp_dim_factor: 2
proj_drop: 0.1
transformer_drop_rate: 0.2
transformer_attn_drop: 0.4
final_dropout: 0.5
head_lr: 0.0001384466273952018
finetune_lr: 3.103788942300277e-05
Score: 0.9004563689231873

Trial 1 summary
Hyperparameters:
embedding_dim: 256
num_heads: 8
num_blocks: 4
mlp_dim_factor: 4
proj_drop: 0.2
transformer_drop_rate: 0.2
transformer_attn_drop: 0.30000000000000004
final_dropout: 0.4
head_lr: 0.0001319548679279438
finetune_lr: 3.048649562411296e-06
Score: 0.8809218406677246

Trial 2 summary
Hyperparameters:
embedding_dim: 192
num_heads: 4
num_blocks: 6
mlp_dim_factor: 4
proj_drop: 0.2
transformer_drop_rate: 0.4
transformer_attn_drop: 0.30000000000000004
final_dropout: 0.5
head_lr: 0.000435909444

In [ ]:
#Build the FINAL Best Model
print("Building the best model with the optimal hyperparameters...")
#best_hps variable comes from running Cell 9
final_model = build_hyper_model(best_hps)
final_model.summary()

#Define Callbacks for FINAL Training
#We use the full, original patience settings here
FINAL_MODEL_PATH = os.path.join(DATA_ROOT, "eanet_skin_model_FINAL_TUNED.keras")
final_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.2, patience=4, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(FINAL_MODEL_PATH, monitor="val_auc", mode="max", save_best_only=True)
]

#STAGE 1: Train the Head
print("\nFinal Training: STAGE 1 (Head)")
#Getting the winning head_lr from best_hps
final_head_lr = best_hps.get('head_lr')

final_model.get_layer("transformer_backbone").trainable = False
final_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=final_head_lr, weight_decay=1e-4),
    loss=loss_fn, #loss_fn from Cell 6
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)
start_time_stage1 = time.time()
history_head = final_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    verbose=1
)
end_time_stage1 = time.time()

#STAGE 2: Fine-Tuning
print("\nFinal Training: STAGE 2 (Fine-Tune)")
#Getting the winning finetune_lr from best_hps
final_finetune_lr = best_hps.get('finetune_lr')

final_model.get_layer("transformer_backbone").trainable = True
final_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=final_finetune_lr, weight_decay=1e-4),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)
start_time_stage2 = time.time()
history_fine_tune = final_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=70,
    callbacks=final_callbacks,
    initial_epoch=len(history_head.history['loss']),
    verbose=1
)
end_time_stage2 = time.time()

#Final Evaluation on Test Set
#The ModelCheckpoint callback will have saved the best model.
#We load it back to ensure we evaluate the *very best* version.
print(f"\nLoading best saved final model from: {FINAL_MODEL_PATH}")
loaded_best_model = tf.keras.models.load_model(
    FINAL_MODEL_PATH,
    custom_objects=custom_objects
)

print("\nEvaluating the final tuned model on the test set...")
test_results = loaded_best_model.evaluate(test_ds, return_dict=True)

print("\nFinal Test Set Evaluation Results")
precision = 0.0
recall = 0.0
for metric, value in test_results.items():
    print(f"{metric}: {value:.4f}")
    if metric == "precision":
        precision = value
    if metric == "recall":
        recall = value

#Manually calculate F1 Score
if precision + recall > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)
    print(f"f1_score (calculated): {f1_score:.4f}")
else:
    print("f1_score (calculated): 0.0")

#Print Stats
print("\nFinal Model Stats")
if os.path.exists(FINAL_MODEL_PATH):
    file_size_bytes = os.path.getsize(FINAL_MODEL_PATH)
    file_size_mb = file_size_bytes / (1024 * 1024)
    print(f"Model Size on Disk: {file_size_mb:.2f} MB")

total_time_sec = (end_time_stage1 - start_time_stage1) + (end_time_stage2 - start_time_stage2)
total_epochs_ran = len(history_head.history['loss']) + len(history_fine_tune.history['loss'])
avg_time_per_epoch_sec = total_time_sec / total_epochs_ran
print(f"Total Training Time: {total_time_sec / 60:.2f} minutes")
print(f"Total Epochs Trained: {total_epochs_ran}")

Building the best model with the optimal hyperparameters...


Model: "eanet_hyper_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_extractor                 │ (None, 7, 7, 3072)     │             0 │
│ (PatchExtractor)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 49, 3072)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 49, 256)        │       786,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add (Add)                       │ (None, 49, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 49, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_backbone            │ (None, 49, 256)        │     7,895,040 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_12          │ (None, 49, 256)        │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,683,011 (33.12 MB)

 Trainable params: 8,683,011 (33.12 MB)

 Non-trainable params: 0 (0.00 B)


Final Training: STAGE 1 (Head)
Epoch 1/15
115/115 ━━━━━━━━━━━━━━━━━━━━ 83s 425ms/step - acc: 0.5686 - auc: 0.5545 - loss: 1.3060 - precision: 0.2242 - recall: 0.4985 - val_acc: 0.7208 - val_auc: 0.8022 - val_loss: 1.0490 - val_precision: 0.3918 - val_recall: 0.7804
Epoch 2/15
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 149ms/step - acc: 0.6601 - auc: 0.6864 - loss: 1.1154 - precision: 0.3059 - recall: 0.5941 - val_acc: 0.7183 - val_auc: 0.8123 - val_loss: 1.0976 - val_precision: 0.3901 - val_recall: 0.7884
Epoch 3/15
115/115 ━━━━━━━━━━━━━━━━━━━━ 18s 158ms/step - acc: 0.6902 - auc: 0.7215 - loss: 1.0725 - precision: 0.3412 - recall: 0.6430 - val_acc: 0.7678 - val_auc: 0.8155 - val_loss: 1.0220 - val_precision: 0.4430 - val_recall: 0.7407
Epoch 4/15
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - acc: 0.7028 - auc: 0.7414 - loss: 1.0381 - precision: 0.3552 - recall: 0.6499 - val_acc: 0.7399 - val_auc: 0.8289 - val_loss: 0.9941 - val_precision: 0.4153 - val_recall: 0.8175
Epoch 5/15
115/115 ━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'transformer_block_2', howev


Evaluating the final tuned model on the test set...
24/24 ━━━━━━━━━━━━━━━━━━━━ 314s 13s/step - acc: 0.8423 - auc: 0.9191 - loss: 0.9537 - precision: 0.5638 - recall: 0.7305

Final Test Set Evaluation Results
acc: 0.8329
auc: 0.9216
loss: 0.9914
precision: 0.5565
recall: 0.7079
f1_score (calculated): 0.6231

Final Model Stats
Model Size on Disk: 99.98 MB
Total Training Time: 28.26 minutes
Total Epochs Trained: 70
